In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import hpelm
from hpelm import ELM
import cv2
import os
import h5py
import time
import pickle
import joblib

### loading binary model for testing

In [2]:
# Load the pre-trained model from an H5 file
model = load_model('binary.h5')

# Load and preprocess the image using OpenCV
img_path = 'sad.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))
img_array = np.expand_dims(img, axis=0)  # Reshape to (1, 224, 224, 3)
img_array = img_array / 255.0  # Normalize the image

# Predict the class of the image
predictions = model.predict(img_array)
print(predictions)

1/1 [==============================] - 1s 1s/step
[[0.07690308 0.9230969 ]]


### training X, Y features

In [3]:
train_path = 'KDEF/Train/'
test_path='KDEF/Test/'
cates = ['Stress','NoStress']

In [4]:
def load_images_and_labels(data_path, cates): 
  X = []
  y = []
  i = 0
  for index, cate in enumerate(cates): 
    for img_name in os.listdir(data_path + cate + '/'):
      i = i +1
      #print(i)
      img = cv2.imread(data_path + cate + '/' + img_name)
      if img is not None: 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = Image.fromarray(img, 'RGB')
        
        # resize image to 227x227 which is the required input size of the Alexnet model
        img_rs = img_array.resize((224,224))
        
        # convert the image to array
        img_rs = np.array(img_rs)
      
        X.append(img_rs)
        y.append(index)
  return X, y

In [5]:
def preprocess_data(X, y):
  # convert X from list to array
  X = np.array(X)
  # convert integer values of X into floats
  X = X.astype(np.float32)
  # normalization 
  X = X/255.0
  # one-hot encoding the labels 
  y = to_categorical(np.array(y))
    
  return X, y

In [6]:
X_train, y_train = load_images_and_labels(train_path, cates)
X_test, y_test = load_images_and_labels(test_path, cates)

In [7]:
X_train_features, y_train_labels = preprocess_data(X_train, y_train)

In [8]:
X_test_data, y_testing_labels=preprocess_data(X_test, y_test)

### Model Generate

In [9]:
def elm_model_generate(data_train, training_data,ELM_HIDDEN_NEURONS,activation_function):
    NUM_CLASS=2
    #ELM_HIDDEN_NEURONS=250
    #target_train_oh = np_utils.to_categorical(training_data.classes, NUM_CLASS)
    
    elm_model = hpelm.elm.ELM(data_train.shape[1], NUM_CLASS)
    elm_model.add_neurons(ELM_HIDDEN_NEURONS, func=activation_function)
    #print(data_train,target_train_oh)
    start_time=time.time()
    elm_model.train(data_train, training_data, 'c')
    stop_time=time.time()
    print("training time of ELM Model for ",stop_time-start_time)
    
    return elm_model

##### using serializable functions not hpelm

In [10]:
from keras.applications import ResNet50
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import SGD

def cnn_generate(X_train, y_train, layer_name):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D(name=layer_name))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  # num_classes is the number of classes in your dataset

    # Compile the model
    model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)  # Adjust epochs and batch_size as needed
    
    # Extract features using the hidden layer model
    hidden_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    cnn_train_result = hidden_layer_model.predict(X_train)
    
    # Print the summary of the main model
    model.summary()

    return hidden_layer_model, cnn_train_result


In [16]:
def elm_model_generate_modified(data_train, training_data, ELM_HIDDEN_NEURONS, activation_function):
    NUM_CLASS = 2
    elm_model = hpelm.ELM(data_train.shape[1], NUM_CLASS)
    elm_model.add_neurons(ELM_HIDDEN_NEURONS, func=activation_function)
    
    start_time = time.time()
    elm_model.train(data_train, training_data, 'c')
    stop_time = time.time()
    print("training time of ELM Model for ", stop_time - start_time)
    
    # Extract model parameters
    model_params = {
        "input_size": elm_model.inputs,
        "output_size": elm_model.outputs,
        "weights": elm_model.nnet[0].W,  # Assuming single layer ELM
        "biases": elm_model.nnet[0].b,   # Assuming single layer ELM
        "activation_function": activation_function,
        "ELM_HIDDEN_NEURONS": ELM_HIDDEN_NEURONS
    }
    
    return model_params

In [11]:
def train_model(model, X_train, y_train, X_test, y_test, epochs):
  # Data generator
  datagen = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)
  # iteration on the training set
  batch_size = 16
  it_train = datagen.flow(X_train, y_train,batch_size )
  # path to save checkpoint 
  path_cp = os.getcwd() + '/' + 'weights_.hdf5'
  checkpoint_ = ModelCheckpoint(path_cp, monitor = 'loss', save_best_only = True, mode = 'auto')
  steps = X_train.shape[0]//batch_size
  # Fitting the model
  history = model.fit(it_train, epochs = epochs, steps_per_epoch = steps, 
                                validation_data = (X_test, y_test), verbose = 1, 
                                callbacks = checkpoint_)
  # Evaluating the model
  _, acc = model.evaluate(X_test, y_test, verbose = 1)
  print('%.3f' % (acc * 100.0))
  p1=plt
  p1.plot(history.history['accuracy'])
  p1.plot(history.history['val_accuracy'])
  p1.title('model accuracy')
  p1.ylabel('accuracy')
  p1.xlabel('epoch')
  p1.legend(['train', 'test'], loc='upper left')
  p1.show()


  p2=plt
  #training loss vs validation loss
  p2.plot(history.history['loss'])
  p2.plot(history.history['val_loss'])
  p2.title('model loss')
  p2.ylabel('loss')
  p2.xlabel('epoch')
  p2.legend(['train', 'test'], loc='upper left')
  p2.show()
  return history, acc,model

In [12]:
def hidden_layer_generate(cnn_model, X_train, lname):
    layer_name = lname 
    hidden_layer_model = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer(layer_name).output)
    cnn_train_result = hidden_layer_model.predict(X_train)

    return hidden_layer_model, cnn_train_result

In [21]:
resnetModel, modified_cnn_results = cnn_generate(X_train_features, y_train_labels,'global_average_pooling2d_2')
t0=time.time()
train_history, acc ,cnn_model= train_model(resnetModel, X_train_features, y_train_labels, X_test_data, y_testing_labels, 
                                           epochs = 30)
print("training_time", time.time()-t0)
cnn_model.save("resnet50_binary_30.h5")

Epoch 1/10
 39/111 [=========>....................] - ETA: 1:47:05 - loss: 0.5152 - accuracy: 0.7909

KeyboardInterrupt: 

In [13]:
cnn_model = load_model("binary.h5") # load float array [[0. 1.]] model
hidden_layer_model, cnn_train_result = hidden_layer_generate(cnn_model, X_train_features, 'global_average_pooling2d_2')
ELM_HIDDEN_NEURONS=100
activation_function='tanh'
elm_model = elm_model_generate(cnn_train_result, y_train_labels,ELM_HIDDEN_NEURONS,activation_function)

138/138 [==============================] - 246s 2s/step
training time of ELM Model for  0.0837407112121582


In [19]:
elm_model??

Type:        ELM
String form:
ELM with 2048 inputs and 2 outputs
Hidden layer neurons: 100 tanh
File:        c:\users\varun\onedrive\desktop\imagemodel\imagemodel\lib\site-packages\hpelm\elm.py
Source:     
class ELM(object):
    """Interface for training Extreme Learning Machines (ELM).

    Args:
        inputs (int): dimensionality of input data, or number of data features
        outputs (int): dimensionality of output data, or number of classes
        classification ('c'/'wc'/'ml', optional): train ELM for classfication ('c') / weighted classification ('wc') /
            multi-label classification ('ml'). For weighted classification you can provide weights in `w`. ELM will
            compute and use the corresponding classification error instead of Mean Squared Error.
        w (vector, optional): weights vector for weighted classification, lenght (`outputs` * 1).
        batch (int, optional): batch size for data processing in ELM, reduces memory requirements. Does not work
  

In [15]:
res = elm_model.predict(cnn_train_result)

In [17]:
cnn_train_result.shape

(4409, 2048)

In [21]:
res[1]

array([0.9051452 , 0.09613691])

In [24]:
load_elm_model = pickle.load(open('load_elm_model.pkl', 'rb'))

In [30]:
random_data = np.random.rand(4409, 2048)
load_res = load_elm_model.predict(random_data)

In [32]:
load_res.shape

(4409, 2)

In [33]:
load_res[1]

array([0.21534602, 0.79157454])

In [96]:
def load_elm_model(filename):
    with h5py.File(filename, 'r') as hf:
        print('List of arrays in this file: \n', list(hf.keys()))
        input_weights = hf['input_weights'][:]
        beta = hf['beta'][:]
        biases = hf['biases'][:]

    input_size = input_weights.shape[1]
    hidden_size = input_weights.shape[0]
    output_size = beta.shape[1]
    elm = ELM(input_size, output_size)
    elm.input_weights = input_weights
    elm.biases = biases
    elm.beta = beta
    return elm

load_elm_model = load_elm_model('elm_model.h5')

# Print to verify
# print("Input Weights:", input_weight.shape)
# print("Beta:", beta.shape)
# print("Biases:", biases.shape)

List of arrays in this file: 
 ['beta', 'biases', 'input_weights']


In [16]:
import numpy as np
np.__version__

'1.24.4'

In [22]:
def inspect_nnet(elm):
    nnet = elm.nnet
    print("Attributes of nnet:")
    for attr in dir(nnet):
        if not attr.startswith('_'):
            print(attr)

    print("\nContents of nnet:")
    for attr in dir(nnet):
        if not attr.startswith('_'):
            print(f"{attr}: {getattr(nnet, attr)}")

In [19]:
def print_elm_attributes(elm):
    for attr in dir(elm):
        if not attr.startswith('_'):
            print(attr)

In [63]:
elm_model.nnet.B.shape

(100, 2)

In [15]:
cnn_train_result.shape

(4409, 2048)

### convert into pickle file

In [49]:
hidden_layer_model.save("hidden_layer_model.h5")

c:\Users\varun\OneDrive\Desktop\imageModel\imagemodel\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [51]:
hidden_layer_model_pkl = load_model("hidden_layer_model.h5")
hidden_layer_model_pkl??

Signature:   hidden_layer_model_pkl(*args, **kwargs)
Type:        Functional
String form: <keras.src.engine.functional.Functional object at 0x000001B2A4DC6E20>
File:        c:\users\varun\onedrive\desktop\imagemodel\imagemodel\lib\site-packages\keras\src\engine\functional.py
Source:     
class Functional(training_lib.Model):
    """A `Functional` model is a `Model` defined as a directed graph of layers.

    Three types of `Model` exist: subclassed `Model`, `Functional` model,
    and `Sequential` (a special case of `Functional`).
    In general, more Keras features are supported with `Functional`
    than with subclassed `Model`s, specifically:

    - Model cloning (`keras.models.clone`)
    - Serialization (`model.get_config()/from_config`, `model.to_json()`
    - Whole-model saving (`model.save()`)

    A `Functional` model can be instantiated by passing two arguments to
    `__init__`. The first argument is the `keras.Input` Tensors that represent
    the inputs to the model. The s

In [53]:
hidden_layer_model_pkl.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50_input (InputLayer  [(None, 224, 224, 3)]     0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
Total params: 23587712 (89.98 MB)
Trainable params: 23534592 (89.78 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [18]:
import joblib
new_elm = joblib.load('newElm.pkl')